### import and implement model

In [7]:
# packages
import torch.nn as nn
import torch.optim as optim
from torch.nn import Conv2d, LeakyReLU, MaxPool2d, Linear # import them seperetly because I think its more readable
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split
import pandas as pd
import numpy as np
import os 
import gc
import torch

In [13]:
# set wd
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/PyTorch/Fracture')

import data and load onto tensors

In [12]:
# image dataset class
class ImageDataset():
    def __init__(self, class_dir, img_dir): # load labels and the img 
        self.img_labels = pd.read_csv(class_dir)
        self.img_dir = img_dir

    def __len__ (self): # len of labels for image
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])  
        image = read_image(img_path)  # read image
        label = self.img_labels.iloc[idx, 1] # read label
        return image, label


In [ ]:
# directories for classes and images
class_dir = './images/class_ids.csv' 
image_dir = './images/resize_data' 

# load dataset using made class function
data_set = ImageDataset(class_dir, image_dir) # create dataset

# set train and test set
train_size = int(.9 * len(data_set)) # using 70% for training, can introduce more testing samples if testing isnt robust enough
test_size = len(data_set) - train_size 

# random split 
training, testing = random_split(data_set, [train_size, test_size])

In [5]:
# define early, needed sooner then other params
batch_size = 64 # batch size, may have to reduce because of mem constraints
device = "cuda" # use gpu

In [11]:
# load the split data on the tensors
train_loader = DataLoader(training, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing, batch_size=batch_size, shuffle=True)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [13]:
print(torch.cuda.is_available()) # check if gpu is working correctly

True


model without transfer learning (will add just wanted to build one from stratch)

In [8]:
# I chose to use a CNN for the image classifcation. 
# CNNs preform much better then feed forward networks for image classification tasks and are still easy to implement

class CNN (nn.Module):
    def __init__(self):
        super().__init__()
        # 1 input layer, to 32 filters, stride of one pixel, 3x3 kernal, padding = (kernal - 1)/2

        # 3 layers like this
        self.conv1 = Conv2d(in_channels=1, out_channels=32, stride=1, kernel_size=3, padding=1) 
        self.Lrelu1 = LeakyReLU() # better preformance on average compared to regular ReLu
        self.conv2 = Conv2d(in_channels=32, out_channels=32, stride=1, kernel_size=3, padding=1)
        self.Lrelu2 = LeakyReLU()
        self.maxpool1 = MaxPool2d(kernel_size = 2, stride = 2)

        # 3 layers like this
        self.conv3 = Conv2d(in_channels=32, out_channels=64, stride=1, kernel_size=3, padding=1) 
        self.Lrelu3 = LeakyReLU() # better preformance on average compared to regular ReLu
        self.conv4 = Conv2d(in_channels=64, out_channels=64, stride=1, kernel_size=3, padding=1)
        self.Lrelu4 = LeakyReLU()
        self.maxpool2 = MaxPool2d(kernel_size = 2, stride = 2)

        # 3 layers like this
        self.conv5 = Conv2d(in_channels=64, out_channels=128, stride=1, kernel_size=3, padding=1) 
        self.Lrelu5 = LeakyReLU() # better preformance on average compared to regular ReLu
        self.conv6 = Conv2d(in_channels=128, out_channels=128, stride=1, kernel_size=3, padding=1)
        self.Lrelu6 = LeakyReLU()
        self.maxpool3 = MaxPool2d(kernel_size = 2, stride = 2)

        # fully connected layers
        self.fc1 = Linear(100352, 1024) # 100352 = 128 * 28 * 28 | 28 = W/8
        self.relu1 = LeakyReLU()
        self.fc2 = Linear(1024, 512)
        self.relu2 = LeakyReLU()
        self.fc3 = Linear(512, 256)
        self.relu3 = LeakyReLU()
        self.fc4 = nn.Linear(256, 1) # one output

        self.dropout = nn.Dropout(p=0.2) # 20% chance a nueron would be dropped, 
        # this reduces overfitting making one neuron not resposnible for everything, also improves regualrization

    def forward(self, x):
        # Pass through Convolutional Block 1
        x = self.conv1(x)
        x = self.Lrelu1(x)
        x = self.conv2(x)
        x = self.Lrelu2(x)
        x = self.maxpool1(x)

        # Pass through Convolutional Block 2
        x = self.conv3(x)
        x = self.Lrelu3(x)
        x = self.conv4(x)
        x = self.Lrelu4(x)
        x = self.maxpool2(x)

        # Pass through Convolutional Block 3
        x = self.conv5(x)
        x = self.Lrelu5(x)
        x = self.conv6(x)
        x = self.Lrelu6(x)
        x = self.maxpool3(x)

        x = torch.flatten(x, 1)

        # Pass through Fully Connected Layers
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)

        # Pass through the final Linear layer
        x = self.fc4(x)

        # Apply Dropout, sigmoind applied in loss function, better preformance
        x = self.dropout(x) # Apply dropout

        return x


model training loop

In [9]:
# run model on GPU
model = CNN()
model.to(device)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Lrelu1): LeakyReLU(negative_slope=0.01)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Lrelu2): LeakyReLU(negative_slope=0.01)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Lrelu3): LeakyReLU(negative_slope=0.01)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Lrelu4): LeakyReLU(negative_slope=0.01)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Lrelu5): LeakyReLU(negative_slope=0.01)
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Lrelu6): LeakyReLU(negative_slope=0.01)
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1

In [10]:
learning_rate = 0.001# standard learning rate
loss_fn = nn.BCEWithLogitsLoss() # add activation function in here
optimizer = optim.AdamW(model.parameters(), lr=learning_rate) # adam w has better preformance, weight decay is applied sep, 
# leads to more peak ram may have to reduce batch size

In [12]:
# early stopping class
class early_stopping:
    def __init__(self):
        self.patience = 15 # wait 15 rounds, can adjust if needed
        self.steps = 0
        self.loss = float('inf') # so first round it takes loss val
    def stop(self, val_loss):
        if val_loss < self.loss:
            self.loss = val_loss
            self.steps = 0
        if val_loss > self.loss:
            self.steps += 1
        if self.steps >= self.patience: 
            return True
        else:
            return False

stopper = early_stopping()

In [14]:
def training(epochs):
    model.train()  # Set the model to training mode
    training_losses = []  # To track loss history
    for i in range(epochs):
        tloss = 0.0
        n_rounds = 0
        for batch_idx, (image, label) in enumerate(train_loader):
            # Move data to device and ensure correct data types
            image = image.to(device).float()
            label = label.to(device).float()
        
            # Zero gradients
            optimizer.zero_grad()
        
            # Forward pass
            predictions = model(image)
            predictions = predictions.squeeze()
            loss = loss_fn(predictions, label)
        
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Accumulate loss
            tloss += loss.item()
            n_rounds = batch_idx + 1
            avg_loss = tloss / n_rounds
            round(avg_loss, 2)

            # early stopping stats
            min_avg_loss = float('inf')
                        
            # early stopping
            if avg_loss < min_avg_loss:
                min_avg_loss = round(avg_loss, 2)
                round_count = 0
            if min_avg_loss > avg_loss:
                round_count += 1
            if round_count > 2:
                break

        print(f"at epoch {i}, loss is {avg_loss:.4f}") 
    
    return model, tloss



In [15]:
# retrain model / "delete"
torch.cuda.empty_cache()
gc.collect()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [18]:
t_model, tloss = training(4)

at epoch 0, loss is 0.1624
at epoch 1, loss is 0.1925
at epoch 2, loss is 0.1615
at epoch 3, loss is 0.1436


In [19]:
results = [] # array for res
model.eval() # set model to eval mode, disables dropout

# testing loop
for images, labels in test_loader:
    images = images.to(device).float() # load onto device with correct data type
    labels = labels.to(device).float()

    predictions = model(images) # make predections on image in model
    preds_1_0 = torch.where(predictions>0.5, 1, 0)
    correct = (preds_1_0 == labels)
    results.append(correct.detach().cpu().numpy().mean())

accuracy = np.array(results).mean()
print(accuracy)


0.5114248181706114
